# SQL

## Conexión a PostgreSQL usando SQLAlchemy

In [2]:
import os
import pandas as pd
from typing import Union
from sqlalchemy import create_engine, text

# Reemplaza USER y PASSWORD del .env
engine = create_engine("sqlite:///data/sql-murder-mystery.db")

# Probar la conexión
try:
    connection = engine.connect()
    print("Conexión exitosa")
    connection.close()
except Exception as e:
    print(f"Error al conectar: {e}")


def ejecutar_query(query: str) -> Union[pd.DataFrame, str]:
    """
    Ejecuta una consulta SQL en la base de datos y devuelve un DataFrame.

    Parámetros:
    - query (str): La consulta SQL a ejecutar.

    Retorna:
    - Union[pd.DataFrame, pl.DataFrame, str]: Retorna un DataFrame en pandas o polars dependiendo del parámetro 'devolver_polars',
      o un mensaje de éxito/error si la operación no es un SELECT.
    """
    
    with engine.connect() as connection:
        try:
            result = connection.execute(text(query))
            
            # Determinar si es una consulta SELECT para devolver datos
            if result.returns_rows:
                rows = result.fetchall()
                columns = result.keys()

                # Convertir filas en un diccionario de listas para crear correctamente el DataFrame en polars
                data_dict = {col: [row[idx] for row in rows] for idx, col in enumerate(columns)}

                # Verificación de coincidencia de columnas y filas
                return pd.DataFrame(data_dict)
            else:
                connection.commit()
                return f"Query ejecutada correctamente: {result.rowcount} filas afectadas."
        except Exception as e:
            return f"Error al ejecutar la query: {e}"


Conexión exitosa


In [4]:
pd.set_option('display.max_colwidth',None)
query ="""SELECT * 
        FROM crime_scene_report
        WHERE type='murder'
        AND city='SQL City'
        AND date='20180115'"""


ejecutar_query(query=query)

,date,type,description,city
0,20180115,murder,"Security footage shows that there were 2 witnesses. The first witness lives at the last house on ""Northwestern Dr"". The second witness, named Annabel, lives somewhere on ""Franklin Ave"".",SQL City


In [5]:
query = "SELECT * FROM person"

ejecutar_query(query=query)

,id,name,license_id,address_number,address_street_name,ssn
0,10000,Christoper Peteuil,993845,624,Bankhall Ave,747714076
1,10007,Kourtney Calderwood,861794,2791,Gustavus Blvd,477972044
2,10010,Muoi Cary,385336,741,Northwestern Dr,828638512
3,10016,Era Moselle,431897,1987,Wood Glade St,614621061
4,10025,Trena Hornby,550890,276,Daws Hill Way,223877684
...,...,...,...,...,...,...
10006,99936,Luba Benser,274427,680,Carnage Blvd,685095054
10007,99941,Roxana Mckimley,975942,1613,Gate St,512136801
10008,99965,Cherie Zeimantz,287627,3661,The Water Ave,362877324
10009,99982,Allen Cruse,251350,3126,N Jean Dr,348734531


In [6]:
query= "SELECT * FROM person WHERE address_street_name= 'Northwestern Dr'  ORDER BY address_number DESC "

ejecutar_query(query=query)

,id,name,license_id,address_number,address_street_name,ssn
0,14887,Morty Schapiro,118009,4919,Northwestern Dr,111564949
1,17729,Lasonya Wildey,439686,3824,Northwestern Dr,917817122
2,53890,Sophie Tiberio,957671,3755,Northwestern Dr,442830147
3,73368,Torie Thalmann,773862,3697,Northwestern Dr,341559436
4,96595,Coretta Cubie,303645,3631,Northwestern Dr,378403829
5,19420,Cody Schiel,890431,3524,Northwestern Dr,947110049
6,93509,Emmitt Aceuedo,916706,3491,Northwestern Dr,979073160
7,87456,Leonora Wolfsberger,215868,3483,Northwestern Dr,565203106
8,36378,Freddie Ellzey,267882,3449,Northwestern Dr,474117596
9,53076,Boris Bijou,664914,3327,Northwestern Dr,401191868


In [37]:
query= "SELECT * FROM get_fit_now_member WHERE name LIKE '%Annabel% ' "

ejecutar_query(query=query)

,id,person_id,name,membership_start_date,membership_status
0,90081,16371,Annabel Miller,20160208,gold


In [45]:
query= "SELECT * FROM person WHERE name LIKE '%Annabel% ' or address_street_name='Franklin Ave' "

ejecutar_query(query=query)

,id,name,license_id,address_number,address_street_name,ssn
0,12207,Wilmer Wolever,509484,139,Franklin Ave,636825374
1,16371,Annabel Miller,490173,103,Franklin Ave,318771143
2,17683,Johnnie Schee,968887,1277,Franklin Ave,815977821
3,18651,Carleen Etoll,356746,22,Franklin Ave,193369255
4,22636,Zachary Ybarbo,768359,785,Franklin Ave,285346605
5,24737,Gema Nantz,273410,3968,Franklin Ave,180545802
6,30654,Clarita Rickels,418084,2254,Franklin Ave,714941023
7,32264,Shelby Dezeeuw,735415,1391,Franklin Ave,143197463
8,33793,Amado Mattan,161915,99,Franklin Ave,125205748
9,34592,Cordell Lindamood,592762,3657,Franklin Ave,509890333


In [40]:
query= "SELECT * FROM interview WHERE person_id='16371'"

ejecutar_query(query=query)

,person_id,transcript
0,16371,"I saw the murder happen, and I recognized the killer from my gym when I was working out last week on January the 9th."


In [46]:
query= "SELECT * FROM INTERVIEW WHERE person_id='14887'"
ejecutar_query(query=query)
	

,person_id,transcript
0,14887,"I heard a gunshot and then saw a man run out. He had a ""Get Fit Now Gym"" bag. The membership number on the bag started with ""48Z"". Only gold members have those bags. The man got into a car with a plate that included ""H42W""."


In [51]:
query= "SELECT* FROM drivers_license WHERE plate_number LIKE '%H42W%'"

ejecutar_query(query=query)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,183779,21,65,blue,blonde,female,H42W0X,Toyota,Prius
1,423327,30,70,brown,brown,male,0H42W2,Chevrolet,Spark LS
2,664760,21,71,black,black,male,4H42WR,Nissan,Altima


In [71]:
query= "SELECT * FROM person WHERE  license_id='183779' OR license_id='423327' OR license_id='664760'"

ejecutar_query(query=query)

,id,name,license_id,address_number,address_street_name,ssn
0,51739,Tushar Chandra,664760,312,Phi St,137882671
1,67318,Jeremy Bowers,423327,530,"Washington Pl, Apt 3A",871539279
2,78193,Maxine Whitely,183779,110,Fisk Rd,137882671


In [119]:
query= "SELECT * FROM get_fit_now_member WHERE name LIKE 'Maxine Whitely	' "
ejecutar_query(query=query)

,id,person_id,name,membership_start_date,membership_status


In [116]:
query= "SELECT * FROM get_fit_now_member WHERE membership_status='gold' AND name='	Maxine Whitely'  "
ejecutar_query(query=query)

,id,person_id,name,membership_start_date,membership_status


In [92]:
query= "SELECT * FROM get_fit_now_member WHERE membership_status='gold' AND name='Jeremy Bowers'  "
ejecutar_query(query=query)

,id,person_id,name,membership_start_date,membership_status
0,48Z55,67318,Jeremy Bowers,20160101,gold


In [93]:
query= "SELECT * FROM interview WHERE 	person_id='67318' "
ejecutar_query(query=query)

,person_id,transcript
0,67318,"I was hired by a woman with a lot of money. I don't know her name but I know she's around 5'5"" (65"") or 5'7"" (67""). She has red hair and she drives a Tesla Model S. I know that she attended the SQL Symphony Concert 3 times in December 2017.\n"


In [97]:
query= "SELECT * FROM drivers_license WHERE gender='female' and car_make='Tesla' and hair_color='red' "
ejecutar_query(query=query)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,202298,68,66,green,red,female,500123,Tesla,Model S
1,291182,65,66,blue,red,female,08CM64,Tesla,Model S
2,918773,48,65,black,red,female,917UU3,Tesla,Model S


In [135]:
query= "SELECT * FROM facebook_event_checkin WHERE event_name='SQL Symphony Concert' and  date like '201712%' "
ejecutar_query(query=query)

,person_id,event_id,event_name,date
0,62596,1143,SQL Symphony Concert,20171225
1,19260,1143,SQL Symphony Concert,20171214
2,58898,1143,SQL Symphony Concert,20171220
3,69699,1143,SQL Symphony Concert,20171214
4,19292,1143,SQL Symphony Concert,20171213
5,43366,1143,SQL Symphony Concert,20171207
6,92343,1143,SQL Symphony Concert,20171212
7,28582,1143,SQL Symphony Concert,20171220
8,28582,1143,SQL Symphony Concert,20171215
9,81526,1143,SQL Symphony Concert,20171202


In [140]:
query= "SELECT * FROM person WHERE id='99716' or id='24556' "
ejecutar_query(query=query)

,id,name,license_id,address_number,address_street_name,ssn
0,24556,Bryan Pardo,101191,703,Machine Ln,816663882
1,99716,Miranda Priestly,202298,1883,Golden Ave,987756388


In [141]:
query= "SELECT * FROM drivers_license WHERE id='202298	' "
ejecutar_query(query=query)

,id,age,height,eye_color,hair_color,gender,plate_number,car_make,car_model
0,202298,68,66,green,red,female,500123,Tesla,Model S


## Solución
El asesino es Jeremy Bowers, sin embargo,  se lo encargo Miranda Priestly